# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a video from your Google Drive

Updated base on https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb

## Choose a video from your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/My Drive/open_pose'

##delete the .DS_Store in each frames folder

In [ ]:
#delete all .DS_Store
import os
os.chdir('data')
!find . -name ".DS_Store" -delete
print(os.getcwd() + '.DS_Store delete successfully')


In [ ]:
# the number of datasets, make sure all datasets are loaded
import os
files = os.listdir('/content/drive/My Drive/open_pose/data')
print(len(files))

## Install OpenPose on Google Colab 


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
# git_repo_url = 'https://github.com/dinatih/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`




##Run openpose, frame2json


In [ ]:
import os

# make dir for json files
def mkdir(path):
	folder = os.path.exists(path)

	if not folder:
		os.makedirs(path)
		print(path)
		print('--- create new folder successful---')
	else:
		print(path)
		print("--- Folder already exists! ---")

#dataset -> each data bags
datasets_path = '/content/drive/My Drive/open_pose/data'
dataset = os.listdir(datasets_path)
print(len(dataset))
for each_dataset in dataset:
	# go to target bag
	dataset_path = datasets_path + '/' + each_dataset
	#make json directory
	json_dir = dataset_path + '/json_file'
	mkdir(json_dir)
	frame_folder = dataset_path + '/frames'
	
	# work on frame.jpg
	!cd openpose && ./build/examples/openpose/openpose.bin --image_dir '{frame_folder}' --render_pose 0 --display 0 --write_json '{json_dir}'


##Run openpose, frame2render_img

In [ ]:
#using -write_image
import os
from os.path import exists, join, basename, splitext
colab_img_path = '/content/drive/My Drive/open_pose/frames'
colab_openpose_image_save_path = '/content/drive/My Drive/open_pose/test_img'
#./build/examples/openpose/openpose.bin --video examples/media/video.avi
#!cd openpose && ./build/examples/openpose/openpose.bin --video '{colab_video_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25
!cd openpose && ./build/examples/openpose/openpose.bin --image_dir '{colab_img_path}' --face --display 0 --write_images '{colab_openpose_image_save_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25
